# Datenanalyse mit pandas

Willkommen! In diesem Notebook lernen wir **pandas** kennen – das wichtigste Werkzeug in Python, wenn es um Daten geht.
Wir arbeiten dabei mit echten **Spotify-Charts-Daten** und finden heraus, wer gerade die Musikwelt dominiert.

## Vorbereitung

Bevor wir loslegen, müssen wir pandas **importieren**. Das ist wie ein Werkzeugkoffer, den wir erst aufmachen müssen.
Die Abkürzung `pd` ist eine Konvention – fast jeder nennt pandas so.

In [ ]:
import pandas as pd

---

## Teil 1: Series und DataFrame – die zwei Bausteine von pandas

pandas hat zwei zentrale Datentypen:

**Series** – Eine einzelne Spalte von Daten. Wie eine Liste, aber mit einem Index.

**DataFrame** – Eine ganze Tabelle, also mehrere Series nebeneinander. Wie ein Excel-Tabellenblatt.

Die Beziehung ist einfach: Wenn du dir eine Spalte aus einem DataFrame herausgreifst, bekommst du eine Series zurück.

In [ ]:
# Eine Series ist wie eine einzelne Spalte
noten = pd.Series([1, 3, 2, 1, 4], name="Mathenoten")
print(noten)
print()
print("Typ:", type(noten))

In [ ]:
# Ein DataFrame ist eine ganze Tabelle
klassenbuch = pd.DataFrame({
    "Name":  ["Lina", "Tom", "Mia", "Noah", "Elif"],
    "Mathe": [1, 3, 2, 1, 4],
    "Sport": [2, 1, 1, 3, 2]
})
print(klassenbuch)
print()
print("Typ:", type(klassenbuch))

In [ ]:
# Greife ich eine Spalte heraus, bekomme ich eine Series
nur_mathe = klassenbuch["Mathe"]
print(nur_mathe)
print()
print("Typ:", type(nur_mathe))

---

## Teil 2: Daten einlesen mit `pd.read_csv()`

In der Praxis tippen wir Daten nicht von Hand ein – wir **lesen sie aus Dateien**.
Das häufigste Format ist CSV (Comma-Separated Values). Damit können wir mit einem einzigen Befehl eine ganze Tabelle laden.

Wir laden jetzt die **globalen Spotify-Charts** der aktuellen Woche.

In [ ]:
df = pd.read_csv("data/global-weekly-current-week.csv")

# Die ersten 5 Zeilen anzeigen
df.head()

---

## Teil 3: Erste Erkundung – Was steckt in den Daten?

Bevor wir mit Daten arbeiten, schauen wir sie uns erstmal an. pandas hat dafür ein paar richtig praktische Befehle:

| Befehl | Was er macht |
|---|---|
| `.head()` | Zeigt die ersten 5 Zeilen |
| `.shape` | Gibt die Anzahl Zeilen und Spalten zurück |
| `.columns` | Listet alle Spaltennamen auf |
| `.info()` | Zeigt Datentypen und fehlende Werte |
| `.describe()` | Statistische Zusammenfassung der Zahlenspalten |

In [ ]:
# Wie groß ist unsere Tabelle?
print("Zeilen x Spalten:", df.shape)

In [ ]:
# Welche Spalten gibt es?
print(df.columns.tolist())

In [ ]:
# Welche Datentypen haben die Spalten?
df.info()

In [ ]:
# Statistische Zusammenfassung – nur für Zahlen
df.describe()

**Schau genau hin:** Bei `describe()` siehst du z.B. den Mittelwert (`mean`) der Streams oder den Maximalwert (`max`). Das gibt dir schon ein Gefühl für die Daten, ohne eine einzige Zeile selbst zu berechnen.

---

## Teil 4: Spalten auswählen & Zeilen filtern

### Eine Spalte auswählen
Mit `df["spaltenname"]` greifst du eine einzelne Spalte heraus (du bekommst eine **Series**).

### Zeilen filtern
Du kannst Bedingungen stellen! `df[df["spalte"] > wert]` gibt nur die Zeilen zurück, bei denen die Bedingung wahr ist.

In [ ]:
# Nur die Songnamen anzeigen (die ersten 10)
df["track_name"].head(10)

In [ ]:
# Welche Songs sind schon länger als 40 Wochen in den Charts?
langzeit_hits = df[df["weeks_on_chart"] > 40]
print(f"{len(langzeit_hits)} Songs sind seit über 40 Wochen in den Charts!")
langzeit_hits[["rank", "artist_names", "track_name", "weeks_on_chart"]].head(10)

In [ ]:
# Songs in den Top 10 (rank <= 10) mit mehr als 30 Mio Streams
top_hits = df[(df["rank"] <= 10) & (df["streams"] > 30_000_000)]
top_hits[["rank", "artist_names", "track_name", "streams"]]

**Beachte:** Bei mehreren Bedingungen brauchen wir:
* `&` für **und** (beide Bedingungen müssen wahr sein)
* `|` für **oder** (mindestens eine muss wahr sein)
* Jede einzelne Bedingung muss in **Klammern** stehen!

---

## Teil 5: Gruppieren & Aggregieren mit `.groupby()`

Eine der mächtigsten Funktionen in pandas! Mit `groupby()` kannst du Daten nach einer Kategorie gruppieren und dann zusammenfassen.

Das Prinzip: **Aufteilen → Berechnen → Zusammenfügen**

Beispiel: Wie viele Songs hat jedes Label (source) in den Charts?

In [ ]:
# Anzahl Songs pro Label/Vertrieb
songs_pro_label = df.groupby("source")["track_name"].count()

# Die Top 10 Labels
songs_pro_label.sort_values(ascending=False).head(10)

In [ ]:
# Durchschnittliche Streams pro Label (Top 10)
avg_streams = df.groupby("source")["streams"].mean()
avg_streams.sort_values(ascending=False).head(10)

---

## Teil 6: Neue Spalten erstellen

Oft wollen wir aus vorhandenen Daten etwas Neues berechnen. Das ist der erste Schritt zum **Feature Engineering** – einem zentralen Konzept im Machine Learning.

Eine neue Spalte erstellen geht ganz einfach:
```python
df["neue_spalte"] = irgendeine_berechnung
```

In [ ]:
# Streams in Millionen – leichter zu lesen
df["streams_mio"] = df["streams"] / 1_000_000

# Wie weit ist der Song von seiner besten Platzierung entfernt?
df["abstand_zum_peak"] = df["rank"] - df["peak_rank"]

df[["rank", "artist_names", "track_name", "streams_mio", "peak_rank", "abstand_zum_peak"]].head(10)

In [ ]:
# Trend: Ist der Song gestiegen oder gefallen?
# previous_rank - rank: positiv = gestiegen, negativ = gefallen
df["trend"] = df["previous_rank"] - df["rank"]

# Songs, die am meisten gestiegen sind
aufsteiger = df.sort_values("trend", ascending=False)
aufsteiger[["rank", "artist_names", "track_name", "previous_rank", "trend"]].head(5)

---

## Teil 7: Einfache Visualisierung mit `.plot()`

Bilder sagen mehr als tausend Zahlen! Mit pandas können wir direkt aus einem DataFrame heraus Diagramme erstellen.
Dafür nutzen wir `.plot()` – das benutzt im Hintergrund die Bibliothek **matplotlib**.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Scatterplot: Hängen Wochen in den Charts und Streams zusammen?
df.plot.scatter(x="weeks_on_chart", y="streams_mio",
                title="Wochen in Charts vs. Streams",
                alpha=0.5)
plt.xlabel("Wochen in den Charts")
plt.ylabel("Streams (Mio.)")
plt.show()

In [ ]:
# Balkendiagramm: Top 10 Songs nach Streams
top10 = df.nlargest(10, "streams_mio").set_index("track_name")
top10["streams_mio"].plot.barh(title="Top 10 Songs – Streams (Mio.)")
plt.xlabel("Streams (Mio.)")
plt.tight_layout()
plt.show()

---
---

# Übungsaufgaben – Die deutschen Spotify-Charts

Jetzt bist du dran! Ab hier arbeiten wir mit den **deutschen Charts**.
Lade zuerst die Daten – danach kommen die Aufgaben.

In [ ]:
de = pd.read_csv("data/de-weekly-current-week.csv")
de.head()

### Aufgabe 1: Daten kennenlernen

Nutze die Befehle aus dem Erklär-Teil, um folgende Fragen zu beantworten:

1. Wie viele Songs sind in den deutschen Charts?
2. Welche Spalten hat die Tabelle?
3. Was ist der Durchschnitt und das Maximum der Streams?

In [ ]:
# Dein Code hier


### Aufgabe 2: Ist dein Lieblingskünstler dabei?

Finde heraus, ob dein Lieblingskünstler in den deutschen Charts ist!

Dafür lernst du einen neuen Trick: `.str.contains()` durchsucht eine Text-Spalte nach einem Stichwort.

**Beispiel:**
```python
df[df["artist_names"].str.contains("Taylor", case=False)]
```
`case=False` bedeutet: Groß-/Kleinschreibung ist egal.

**Aufgabe:** Ersetze den Suchbegriff durch deinen Lieblingskünstler!

In [ ]:
# Ersetze "Taylor" durch deinen Lieblingskünstler
lieblings_kuenstler = "Taylor"

ergebnis = de[de["artist_names"].str.contains(lieblings_kuenstler, case=False)]

if len(ergebnis) > 0:
    print(f"Ja! {lieblings_kuenstler} ist {len(ergebnis)}x in den Charts!")
    # Zeige die Songs an – ergänze die passenden Spaltennamen:
else:
    print(f"Leider ist {lieblings_kuenstler} nicht in den deutschen Charts.")

### Aufgabe 3: Filmmusik in den Charts?

Im Film **„K-Pop: Demon Hunters“** gab es einen Song, der eine Zeit lang in den Spotify-Charts auftauchte.
Ist der Song immer noch in den deutschen Charts vertreten?

**Wichtig:** Scrolle nicht einfach durch die Tabelle und suche mit den Augen – das wäre bei 200 Zeilen ziemlich mühsam.
Nutze stattdessen deine neuen pandas-Skills, um die Daten gezielt abzufragen!

*Tipp:* Du kennst schon `.str.contains()` aus der letzten Aufgabe. Überlege dir einen passenden Suchbegriff.

In [ ]:
# Dein Code hier


### Aufgabe 4: Die Dauerbrenner

Welche Songs sind schon am **längsten** in den deutschen Charts?

Dafür brauchst du `.sort_values()`. Damit sortierst du einen DataFrame nach einer Spalte.

```python
df.sort_values(by="spaltenname", ascending=False)
```
* `ascending=False` → absteigend (größter Wert zuerst)
* `ascending=True` → aufsteigend (kleinster Wert zuerst, ist Standard)

**Aufgabe:** Zeige die 15 Songs, die am längsten in den Charts sind. Zeige nur die Spalten `rank`, `artist_names`, `track_name` und `weeks_on_chart`.

In [ ]:
# Dein Code hier


### Aufgabe 5: Wer dominiert die Charts?

Welcher Künstler hat die **meisten Songs** in den deutschen Charts?

Hier lernst du `.value_counts()` kennen – ein super nützlicher Befehl! Er zählt, wie oft jeder Wert in einer Spalte vorkommt.

```python
df["spaltenname"].value_counts()
```

**Aufgabe:** Finde die Top 10 Künstler mit den meisten Chart-Einträgen.

In [ ]:
# Dein Code hier


### Aufgabe 6: Welches Label hat die meisten Streams?

Die Spalte `source` verrät, welches Label bzw. welcher Vertrieb hinter einem Song steht.

**Aufgabe:** Benutze `groupby()` und `.sum()`, um herauszufinden, welches Label in Deutschland die meisten Streams hat. Zeige die Top 10.

*Tipp:*
```python
de.groupby("source")["streams"].sum().sort_values(ascending=False).head(10)
```

In [ ]:
# Dein Code hier


### Aufgabe 7: Durchschnittliche Beliebtheit

**Aufgabe:** Berechne die **durchschnittlichen Streams** aller Songs in den deutschen Charts. Gib das Ergebnis schön formatiert aus.

*Tipp:* Du kannst `.mean()` auf eine einzelne Spalte anwenden. Nutze einen f-String für die Ausgabe:
```python
f"{zahl:,.0f}"
```
Das `:,.0f` fügt Tausender-Punkte ein und zeigt keine Nachkommastellen.

In [ ]:
# Dein Code hier


### Aufgabe 8: Histogramm der Streams

Ein **Histogramm** zeigt dir die Verteilung von Werten. Wie viele Songs haben wenige Streams? Wie viele haben viele?

**Aufgabe:** Erstelle ein Histogramm über die Spalte `streams`.

*Tipp:*
```python
de["streams"].plot.hist(bins=30, title="Verteilung der Streams")
```
Der Parameter `bins` bestimmt, in wie viele Balken die Daten aufgeteilt werden. Probiere verschiedene Werte aus!

In [ ]:
# Dein Code hier


### Aufgabe 9: Wie mächtig sind die Top 3?

Jetzt wird's spannend: Wie viel **Prozent aller Streams** in den deutschen Charts gehen an die **Top 3 Künstler**?

**Schritt für Schritt:**
1. Berechne die **Gesamt-Streams aller Songs**: `de["streams"].sum()`
2. Finde die **Top 3 Künstler** nach Gesamt-Streams (nutze `groupby` + `sum` + `nlargest`)
3. Berechne den Anteil: `top3_streams / gesamt_streams * 100`

*Tipp:* `nlargest(3)` gibt dir die 3 größten Werte einer Series.

In [ ]:
# Dein Code hier


### Aufgabe 10: Newcomer vs. Dauerbrenner

Erstelle eine neue Spalte `kategorie`, die jeden Song als **"Newcomer"** oder **"Dauerbrenner"** einstuft.

Regeln:
* Weniger als 10 Wochen in den Charts → `"Newcomer"`
* 10 oder mehr Wochen → `"Dauerbrenner"`

Dafür kannst du `numpy` benutzen, das wie ein Helfer für pandas funktioniert:

```python
import numpy as np
de["kategorie"] = np.where(de["weeks_on_chart"] < 10, "Newcomer", "Dauerbrenner")
```

**Aufgabe:**
1. Erstelle die Spalte `kategorie`.
2. Zähle mit `value_counts()`, wie viele Newcomer und wie viele Dauerbrenner es gibt.
3. Berechne die durchschnittlichen Streams für beide Kategorien mit `groupby`.

In [ ]:
import numpy as np

# Dein Code hier


### Aufgabe 11: Battle der Genres – Songs mit Featuring

Viele erfolgreiche Songs sind **Kollaborationen** – also Songs, bei denen mehrere Künstler zusammenarbeiten. Das erkennt man daran, dass in der Spalte `artist_names` ein Komma vorkommt (z.B. `"Artist A, Artist B"`).

**Aufgabe:**
1. Erstelle eine neue Spalte `ist_collab`, die `True` ist, wenn ein Komma im Künstlernamen vorkommt.
   *Tipp:* `de["artist_names"].str.contains(",")`
2. Wie viele Songs sind Kollaborationen, wie viele sind Solo-Songs?
3. Haben Kollaborationen im Durchschnitt **mehr oder weniger Streams** als Solo-Songs?

In [ ]:
# Dein Code hier


### Profi-Aufgabe: Streams-Ranking visualisieren

Erstelle ein **horizontales Balkendiagramm** der 10 meistgestreamten Songs in den deutschen Charts.

**Aufgabe:**
1. Wähle die Top 10 Songs nach Streams aus (`.nlargest(10, "streams")`).
2. Setze den `track_name` als Index (`.set_index("track_name")`).
3. Plotte die Streams als horizontalen Barplot (`.plot.barh()`).
4. Gib dem Diagramm einen Titel und beschrifte die Achsen.

In [ ]:
# Dein Code hier


### Profi-Aufgabe: Aufsteiger & Absteiger der Woche

Finde heraus, welche Songs diese Woche den **größten Sprung nach oben** und den **größten Absturz** gemacht haben.

**Aufgabe:**
1. Erstelle eine Spalte `veraenderung`, die berechnet, wie viele Plätze ein Song gestiegen oder gefallen ist: `previous_rank - rank`.
   *(Positiv = gestiegen, negativ = gefallen)*
2. Zeige die **Top 5 Aufsteiger** (größte positive Veränderung).
3. Zeige die **Top 5 Absteiger** (größte negative Veränderung).

*Tipp:* Für die Absteiger kannst du `nsmallest(5, "veraenderung")` verwenden.

In [ ]:
# Dein Code hier


### Profi-Aufgabe: Deutschland vs. Welt

Gibt es Songs, die in Deutschland beliebt sind, aber weltweit nicht?

**Aufgabe:**
1. Lade die globalen Charts in einen zweiten DataFrame: `glob = pd.read_csv("data/global-weekly-current-week.csv")`
2. Finde Songs, die in den deutschen Charts sind, aber **nicht** in den globalen Charts. 
   *Tipp:* Mit `de[~de["uri"].isin(glob["uri"])]` findest du alle deutschen Songs, deren `uri` nicht in den globalen Charts vorkommt. Die Tilde `~` bedeutet "nicht".
3. Wie viele sind es? Zeige die Top 10 davon nach Streams.

In [ ]:
# Dein Code hier


---

## Geschafft!

Du kannst jetzt mit pandas Daten laden, erkunden, filtern, gruppieren, neue Spalten erstellen und visualisieren – das sind die wichtigsten Grundlagen für **Data Science** und **Machine Learning**.

Im nächsten Notebook werden wir diese Skills nutzen, um Daten für ein echtes ML-Modell vorzubereiten!